In [ ]:
""" psf preprocess """

import scipy.io
import scipy.ndimage
import tifffile

psf_load_path = "data/psf.mat"
psf_save_path = "data/psf.tif"
crop = 16

# (1024, 1024, 155) HWD  65nm float64
psf = scipy.io.loadmat(psf_load_path)['FLFPSF']
# (128, 1024, 1024) DWH  65nm float32
psf = psf.astype('float32').transpose(2, 0, 1)[14:-13,:,:]
# ( 64,  512,  512) DWH 130nm float32
psf = scipy.ndimage.zoom(psf, zoom=0.5)
# ( 32,   32,   32) DHW 130nm float32
psf = psf[
    int(psf.shape[0]/2)-crop : int(psf.shape[0]/2)+crop,
    int(psf.shape[1]/2)-crop : int(psf.shape[1]/2)+crop,
    int(psf.shape[2]/2)-crop : int(psf.shape[2]/2)+crop,
]

tifffile.imsave(psf_save_path, psf)

In [ ]:
""" frame preprocess
Convert the Xunamen output 
from (64 512 512)pixel ( 65 130 130)nm 16bit
to   (32 512 512)pixel (130 130 130)nm 16bit
To save storage space.
"""

import torch
import torch.nn.functional as F
import numpy as np
import os
import tifffile
import tqdm

frames_load_fold = "E:/frames"
frames_save_fold = "D:/frames"

frames_list = os.listdir(frames_load_fold)
for index in tqdm.tqdm(range(len(frames_list))):
    # read 16 bit tiff, note that the order of os.listdir is not guaranteed
    frame = tifffile.imread(
        os.path.join(frames_load_fold, frames_list[index])
    ).astype(np.float32)
    frame = torch.from_numpy(frame).float()
    # 64 512 512 -> 32 512 512
    frame = F.interpolate(
        frame.unsqueeze(0).unsqueeze(0), 
        size = (32, 512, 512)
    ).squeeze(0).squeeze(0).half()
    # save 16 bit with file name formation
    tifffile.imwrite(
        "{}/{:05}.tif".format(frames_save_fold, int(frames_list[index][11:-4])),
        frame.numpy()
    )

In [ ]:
""" mlist preprocess
(x, y, z, FWHM_x, FWHM_y, FWHM_z, peak) -> (z, x, y, var_z, var_x, var_y, peak)
x, y, z start index from 1 to 0, i.e., 1-64 to 0-63
z pixel size from 65 to 130, (64, 512, 512) -> (32, 512, 512)
"""

import torch
import os
import h5py
import scipy.io
import tqdm

mlists_load_fold = "D:/hela/coords_20231003"
mlists_save_fold = "D:/hela/mlists"

mlists_list = os.listdir(mlists_load_fold)
for index in tqdm.tqdm(range(len(mlists_list))):
    # read mlist
    try:
        with h5py.File(
            os.path.join(mlists_load_fold, mlists_list[index]), 'r'
        ) as file: mlist = file['storm_coords'][()].T
    except OSError:
        _, mlist = scipy.io.loadmat(
            os.path.join(mlists_load_fold, mlists_list[index])
        ).popitem()
    mlist = torch.from_numpy(mlist).float()

    # (x, y, z, FWHM_x, FWHM_y, FWHM_z, peak) -> 
    # (z, x, y, FWHM_z, FWHM_x, FWHM_y, peak)
    mlist = mlist[:, [2, 0, 1, 5, 3, 4, 6]]
    # index from (0 - 64) to (-0.5 - 63.5)
    mlist[:, 0:3] = mlist[:, 0:3] - 0.5
    # (65, 130, 130)nm -> (130, 130, 130)nm
    mlist[:, 0] = (mlist[:, 0] + 0.5) / 2 - 0.5
    mlist[:, 3] =  mlist[:, 3] / 2
    # (x, y, z, FWHM_x, FWHM_y, FWHM_z, peak) -> 
    # (z, x, y, var_z, var_x, var_y, peak)
    mlist[:, 3:6] = (mlist[:, 3:6] / 2.355) ** 2

    # save
    scipy.io.savemat(
        "{}/{:05}.mat".format(mlists_save_fold, int(mlists_list[index][11:-4])),
        {"storm_coords": mlist.numpy()}
    )

In [ ]:
""" mlist2label 
"""

import torch
from torch import Tensor
import numpy as np
import os
import tifffile
import h5py
import scipy.io
import tqdm

# dimensional config
dim_frame = Tensor([ 32, 512, 512]).int()   # (C, H, W), (130, 130, 130)nm
up_sample = Tensor([  4,   8,   8]).int()   # (C, H, W)
dim_label = dim_frame * up_sample
# data path
data_fold = "C:/Users/tianrui/Desktop/tubulin/"
frames_load_fold = data_fold + "frames"
mlists_load_fold = data_fold + "mlists"
data_save_fold   = data_fold + "8-(00-32-00-32)-ml-tempaverage-RCC"
temp_save_fold   = data_fold + "tempaverage"
method = "RCC"  # DCC, MCC, or RCC

# file name list
frames_list = os.listdir(frames_load_fold)
mlists_list = os.listdir(mlists_load_fold)
# create folder
if not os.path.exists(data_save_fold): os.makedirs(data_save_fold)

# drift
drift  = torch.from_numpy(np.loadtxt(
    os.path.join(temp_save_fold, "{}.csv".format(method)), delimiter=','
)).float()
drift *= up_sample / Tensor([4, 4, 4]).int()
drift  = drift.round().int()

result = torch.zeros(*dim_label.tolist())
for index in tqdm.tqdm(
    range(len(frames_list)), desc=data_save_fold, unit="frame"
):
    mlist = None
    try:
        with h5py.File(os.path.join(
            mlists_load_fold, mlists_list[index]
        ), 'r') as file: mlist = file['storm_coords'][()].T
    except OSError:
        _, mlist = scipy.io.loadmat(
            os.path.join(mlists_load_fold, mlists_list[index])
        ).popitem()
    mlist = torch.from_numpy(mlist).float()

    # up sample the molecular list
    mlist[:, 0:3]  = (mlist[:, 0:3] + 0.5) * up_sample - 0.5
    # drift correction for molecular list
    mlist[:, 0:3] += drift[index]
    # round and clip
    mlist[:, 0:3]  = torch.round(mlist[:, 0:3])
    mlist[:, 0:3]  = torch.clip(
        mlist[:, 0:3], torch.zeros_like(dim_label), dim_label - 1
    )

    # remove false fitting
    mlist[mlist[:, 6] > 50, 6] = 0

    # label
    for i in range(mlist.shape[0]):
        mean = mlist[i, 0:3]
        peak = mlist[i, 6]
        #peak = torch.clip(mlist[i, 6], 0, 1)
        result[tuple(mean.int())] += peak

    # save after combine 1, 2, 4, 8, 16... frames
    #current_frame = index + 1
    #if current_frame & (current_frame - 1) == 0 or index == len(frames_list)-1:
    #    tifffile.imwrite(
    #        "{}/{:05}.tif".format(data_save_fold, current_frame),
    #        result.numpy()
    #    )
tifffile.imwrite(
    "{}/{:05}.tif".format(data_save_fold, len(frames_list)),
    result.numpy()
)

In [1]:
""" concatenate two or more 3D subframes into a 3D frame 
"""

import numpy as np
import tifffile
import os

LT2RD_fold = [
    "D:/8-(24-29-00-01)-e10-average-RCC-3D", 
    "D:/8-(24-29-01-02)-e10-average-RCC-3D",
    "D:/8-(24-29-02-03)-e10-average-RCC-3D",
    "D:/8-(24-29-03-04)-e10-average-RCC-3D",
    "D:/8-(24-29-04-05)-e10-average-RCC-3D",
    "D:/8-(24-29-05-06)-e10-average-RCC-3D",
    "D:/8-(24-29-06-07)-e10-average-RCC-3D",
    "D:/8-(24-29-07-08)-e10-average-RCC-3D",
    "D:/8-(24-29-08-09)-e10-average-RCC-3D",
    "D:/8-(24-29-09-10)-e10-average-RCC-3D",
]
FN_fold = "D:/8-(24-29-00-10)-e10-average-RCC-3D"  # final
axis = 2    # 1 for vertical, 2 for horizontal

if not os.path.exists(FN_fold): os.makedirs(FN_fold)
for i in range(len(os.listdir(LT2RD_fold[0]))): 
    tifffile.imwrite(
        os.path.join(FN_fold, os.listdir(LT2RD_fold[0])[i]),
        np.concatenate([
            tifffile.imread(
                os.path.join(LT2RD_fold[j], os.listdir(LT2RD_fold[j])[i])
            ) for j in range(len(LT2RD_fold)) 
        ], axis=axis)
    )

In [ ]:
""" put subframes into whole frame
"""

import numpy as np
import tifffile
import os
import tqdm

rng_sub_user = [0, 1, 26, 27, 18, 22]

dim_label = np.array([32*4, 512*8, 512*8]).astype(int)
frame_fold = f"D:/8-({rng_sub_user[2]:02d}-{rng_sub_user[3]:02d}-{rng_sub_user[4]:02d}-{rng_sub_user[5]:02d})-e10-(08-12-09-13)-RCC-3D"
whole_fold =  "C:/Users/tianrui/Desktop/8-(00-32-00-32)-e10-(08-12-09-13)-RCC-3D"

for i in tqdm.tqdm(range(len(os.listdir(frame_fold)))):
    frame_path = os.path.join(frame_fold, os.listdir(frame_fold)[i])
    whole_path = os.path.join(whole_fold, os.listdir(frame_fold)[i])
    frame = tifffile.imread(frame_path).astype(np.float32)
    if os.path.exists(whole_path):
        whole = tifffile.imread(whole_path).astype(np.float32)
    else:
        raise FileNotFoundError
        whole = np.zeros(dim_label).astype(np.float32)
    whole[
        rng_sub_user[0]*128 : rng_sub_user[1]*128,
        rng_sub_user[2]*128 : rng_sub_user[3]*128,
        rng_sub_user[4]*128 : rng_sub_user[5]*128,
    ] = frame
    if not os.path.exists(whole_fold): os.makedirs(whole_fold)
    tifffile.imwrite(whole_path, whole)